## By Albert S. (Pete) Kyle

## BUFN400 Fall 2022---University of Maryland


In [1]:

import os
GLOBAL_NUM_THREADS = "1"
os.environ["OMP_NUM_THREADS"] = GLOBAL_NUM_THREADS
os.environ["OPENBLAS_NUM_THREADS"] = GLOBAL_NUM_THREADS
os.environ["MKL_NUM_THREADS"] = GLOBAL_NUM_THREADS
os.environ["VECLIB_MAXIMUM_THREADS"] = GLOBAL_NUM_THREADS
#os.environ["NUMEXPR_MAX_THREADS"] = GLOBAL_NUM_THREADS
os.environ["NUMEXPR_NUM_THREADS"] = GLOBAL_NUM_THREADS
os.environ["NUMBA_NUM_THREADS"] = GLOBAL_NUM_THREADS

import pandas as pd
import numpy as np
import scipy
import scipy.special
import scipy.optimize
#import scipy.sparse.linalg
import matplotlib
import matplotlib.pyplot as plt

import nbconvert
# import pyppeteer

import sys
import datetime
import math
import string
import time
import timeit
import io
from pprint import pprint
#import numba
#import numexpr

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('NumPy version ' + np.__version__)
print('SciPy version ' + scipy.__version__)
print('matplotlib version ' + matplotlib.__version__)
#print('numba version ' + numba.__version__)

timestamp = datetime.datetime.now().strftime('%Y-%m%d-%H%M')
print("Timestamp:", timestamp)


Python version 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:27:59) [MSC v.1937 64 bit (AMD64)]
Pandas version 2.2.0
NumPy version 1.26.3
SciPy version 1.12.0
matplotlib version 3.8.2
Timestamp: 2024-0908-2141


# Money, Time, and Units of Measurement

Applying the time-value-of-money formulas to practical situations often involves manipulating dates, converting days to years, and makin sure calculations are dimensionally consistent.  In comparison with relative "clean" finance theory, practical calculations can be messy and error-prone.  

Here is a discussion of issues related to dates, time, and units of measurement.

## Dimensions and units of measurement in Physics, Economics, Finance, and Accounting

In physics, mass, distance, and time mass are fundamental dimensions in terms of which quantities are described. Quantities are measured in terms of corresponding units such as kilograms, meters, and seconds. 

Quantities with different units of measurement can be multiplied and divided. Consider, for example, Einstein's physics formula $E = m C^2$. Since mass $m$ is measured in kilograms and speed $C$ is measured in meters per second, energy $E$ must have units $\text{kg }\text{m}^2/\text{sec}^2$. This implies that a car crashing into a brick wall at 60 miles per hour releases four times as much energy as a car crashing at 30 miles per hour.

Quantities with different units of measurement cannot be added or subtracted. You cannot add apples and oranges and obtain a meaningful answer; instead, you are making a mistake. When quantities are added or subtracted, they must have the same units. Like quantities in physics,apples and oranges can be multiplied and divided. It is perfectly meaningful to say, "The exchange rate between apples and oranges is 2.5 apples per orange".

In finance, time and value are key dimensions.

Time is measured in many different units depending on the context. Microseconds are used for high frequency trading, days are often used for stock returns, and interest rates are often measured per year (units of $1/\text{year}$), even though interest accumulates one day at a time. 

Value is typically measured in currency units, such as  U.S. dollars, Euros, Japanese yen, or Chinese yuan.

## Stock and Flows

Economists like to make a distinction between **stocks** and **flows**. In this usage, the word "stocks" means quantities without time units, not "common stock" like in finance.  The term "flows" refers to quantities with time units (e.g. $1 / \text{year}$. This distinction is important in economics, finance, and accounting. Examples of stocks are inventories, assets, and liabilities. Examples of flows are GDP (dollars per year), earnings, sales, revenues, trading volume, and interest rates.  Many financial ratios have time units.  For example, a price-to-earnings ratio, or **P/E** ratio, has units of years, and an earnings-to-price ratio has units "per year", like interest rates. 

1. "Trading volume" is calculated by dividing a quantity traded by an interval of time. 

2. "Dividend rate," is calculated by divided a dollar amount of a dividend by a time interval, then dividing by a stock price. For example, consider a stock which pays a dividend of 1 dollar every quarter and has a price of 100 dollars. The dividend rate is 4 dollars per share per year.  The dividend yield, obtained by dividing the dividend rate by the stock price, is 4 percent per year or 0.04 per year.  The dividend yield has exactly the same units as an interest rate.  Therefore, it makes sense to subtract an interest rate from a dividend yield.  If the interest rate is 3.00 percent, it makes sense to say that the difference between the dividend yield and the interest rate is 1 percent per year. 

3. Similarly, "Earnings" is calculated by dividing "dollar profit" by an interval of time (quarter, year). Earnings has units of "dollars per year" (if for the entire company) or "dollars per share per year". Dividing by the stock price results in an "earnings yield" (reciprocal of price-earnings ratio) with the same time units as an interest rate and with a dividend yield. Thus, it also makes sense to subtract an earnings yield from an interest rate or a dividend yield.

## Real versus Nominal

TBA

## Terminology and Units for Time and Money

This note is about the time value of money.  The key units are **time** and **money**.  Units often get combined with **percent** and **basis points**.

#####  Terminology for Money

Economics and finance courses typically teach that money can be a **medium of exchange**, a **store of value**, or a **unit of account**:

1. Medium of exchange: I pad 5000 dollars for 100 shares of stock XYZ.

2. Store of value: I have a bank account with 5000 dollars in it.

3. Unit of account: My assets are 5000 dollars, and my liabilities are 1000 dollars.

Around the world, there are many different currencies used as money. Practitioners and electronic datasets often use official three-charactoer codes to keep track of them: USD for U.S. dollars, CAD for Canadian dollars, AUD for Australian dollars, EUR for Euros, JPY for Japanese yen, GBP for British pound, CNY for Chinese yuan, RUB for Russian ruble, etc.

The three-character codes for currencies are analogous to the official definitions of weights an measures, with "kg" meaning kilogram, "s" meaning seconds, and "m" meaning meters.

The term "dollar" is ambiguous.  It could mean USD, CAD, AUD, or something else.  In this note, I will use "dollar" as a generic term for a currency when the specific currency does not matter for teaching purposes. For specific currencies, the three-digit code should be used, especially when the exchange rate is relevant.

NB: In real-world applications, making mistakes with currency units, especially dollars, is quite common.  For example, if the exchange rate between USD and CAD is close to **parity** (one USD costs 1.00 CAD and vice versa), then it is easy to confuse one currency with the other by referring to it as "dollars".

##### Terminology for **interest rate**:

1. The phrase "6.00 percent per year" means "0.06 per year" because "percent" means "per one hundred", which is equivalent to "units of $1/100$" or simply "times $1/100$.

2. Interest rates have time units. Here the time units are "per year" or "$1 / \text{year}$, but the same interest rate might be expressed as 3.00 percent per half-year or 0.50 percent per month, which means the same thing as 50 "**basis points**" per month.

All of this is true, even though the fundamental unit for interest rates is days, not months or years!

NB: Units in finance can be confusing because practitioners and academics use terminology in an inconsistent manner. 

## Terminology **percent** and **basis point**. 

In finance:

1. The term **percent** should mean "units of 1/100" or "times 0.01", without time units.  It should be wrong to say that, "The interest rate is 6.00 percent" (without time units) instead of correctly saying "The interest rate is 6.00 percent per year" or "The interest rate is 6.00 percent per annum." Omitting the "per year" should be understood as an abbreviation. In practice, interest rates are always quoted as "**annualized**" rates.

2. The term **basis point**: has two different meanings. It always implies "one percent of one percent", which means units of $1/10000$ or $1 / 10^4$ or $10^{-4}$ or 0.0001, but in some contexts it has time units and in other contexts it does not.  For example, in the statement, "The banker's interest rate spread between a loan with interest rate 7.50 percent per year and a deposit with interest rate 6.00 percent per year is 150 basis points" suggests that basis points mean "units of $1/10000$ per year" (note the implied time units).  But in the statement, "The bid price is 99.75 dollars and the ask price is 100.25 dollars, so the bid-ask spread is 50 basis points"  the 50 basis points has neither time units nor dollar units. 

$$\frac{100.25 - 99.75}{\tfrac{1}{2}(99.75 + 100.25)} = \frac{0.50 \text{ dollars}}{100.00 \text{ dollars}} = \frac{50}{10000} = 50 \cdot 10^{-4} = 50 \text{ basis points},
\qquad \text{with no time units (or dollars units)!}
$$

Some traditional financial purists might insist that basis points should always have time units. Nevertheless, the term "basis point" has historically had an even wider set of different meanings in different contexts. For more than 100 years, commodity traders who traded cotten quoted prices like 67.89 cents per pound, where "one basis point" meant 0.01 cents per pound, which is equivalent to "0.0001 dollars per pound".  In this context, the power $10^{-4}$ survives but the basis points have "per pound" weight units also.

## Dimensionless quantities

It is important to recognize when quantities are dimensionless.

Consider the simple expressions $v_0 \cdot (1 + r \cdot \Delta t)$. In this expression, the number 1 is dimensionless. Since dimensionless quantities can only be added to other dimensionless quantities, the quantity $r \cdot \Delta t$ must also be dimensioness. This is indeed true if $r$ and $\Delta t$ are measured with consistent time units which are reciprocals of each other.  For example, it would be accurate to measure $r$ as interest rate "per year" and $\Delta t$ as a fraction of a "year"; in this case, the time units would indeed cancel out.  It would be a mistake to measure $r$ "per year" and $\Delta t$ in "days". Then $r \cdot \Delta t$ would have units "days per year" and could not be added to the dimensionless quantity 1.

Now consider the expression $\mathrm{e}^{r \cdot \Delta t}$, which also arises in analysing the time value of money. The function $\mathrm{e}^x$ requires that $x$ be a dimensionless quantity.  This is obvious from the definition of of $\mathrm{e}^x$ as a power series with different powers of $x$.  The only way for all of these powers of $x$ to have the same units is for $x$ itself to be dimensionless.

## Interest rates: Keeping track of Years, Months, and Days is Messy!

Except in high-inflation situations, interest rates are generally measured in **annualized** (yearly) terms, even when actual interest payments need to be calculated on a semiannual, quarterly, monthly or daily basis.

Historically, financial market practitioners calculated interest over intuitive periods like years and months. Making interest calculations over days is also sometimes needed. 

Unfortunately, there cannot exist a clean way to convert annual rates to months or days. The reason is that years have different number of days (365 vs. 366 for leap years), and months have different numbers of days as well (28, 30, 31). Thus, "one year" or "one month" are ambiguous lengths of time if you want to convert these concepts to days.

This issue creates problems for keeping track of financial contracts. The historical solution, going back to the days of pencil and paper---before computers or even before hand-held calculators or adding machines---was to make approximations.  For example, pretend that a year has 360 days, 12 months of 30 days each.  This approximation makes calculations easier if done by hand, but it creates problems mapping calculations to exact numbers of days.  As a result, financial calculations related to interest rates have historically had some messiness due to these approximations.  This messiness persists until today.

For example:

1. Interest on bank loans is often calculated daily under the assumption that one year has 360 days.

2. Interest on mortgages and some corporate bonds is often calculated under the assumption that one year consists of 12 months of 30 days each.  This creates problems at the end of February and at the end of months with 31 days.

3.  Interest on Treasury notes and bonds is paid semi-annually, with accrued interest prorated daily based on the exact number of days in the semi-annual period.  This exact number varies from one period to the next.

4. Commercial paper and U.S. Treasury bills use a discount rate based on a 360-day year.

5. Savings accounts might pay interest daily based on a 365-day year (or 366-day year in leap years).

This years-months-days issue creates complexity. See, for example, the Wikipedia article on "Day Count Conventions" (https://en.wikipedia.org/wiki/Day_count_convention).

Additional messiness results from the fact that some days are weekends or holidays (**bank holidays**).

## Keeping Track of Time in Python

In Python numbers like float, int, np.float32, np.float64, np.int32, np.int64 do not have units.

Python itself has a *datetime* package which makes in convenient to keep track of year, month, and day.  Instead of this package, we will keep track of time using *numpy* or *pandas*.

The **np.datetime64** and **np.timedelta64** classes are exceptions to the rule that number in Python are dimensionless. The *numpy.datetime64* class keeps track of time as a 64-bit integer and also keeps track separately of whether the integer units are years, months, days, hours, minutes, seconds, milliseconds, microseconds, or nanoseconds. The integer records the number of time units which have passed since the **Unix epoch**, January 1, 1970  The *numpy.timedelta64* class keeps track of time intervals using 64-bit integers in the same way.

The *pandas* package has its own class for keeping track of time.  These classes build on the *numpy* classes but adds additional functionality, such as keeping track of timezones.

In finance, it is often important to use timezone **aware** dates. When timezone aware dates are not needed, pandas can use **numpy.datetime64**, which is not timezone aware.


## Advice

Inconsistencies in units of measurement often indicate either sloppy thinking or outright errors.

1. A good way to locate mistakes in code or mathematical calculations is to check for consistency of units of measurement.

2.  One often sees expressions involving interest rates expressed in a dubious manner, such as $(1 + r)^N$. Technically, this expression is incorrect if $r$ has units "per year". I suggest always including a time interval, measured in the same units, so the the expression becomes $(1 + r \cdot \Delta t)^N$, where $\Delta t$ has units of years (even if $\Delta t$ is exactly one year!).  These notes use notation $m$ for $\Delta t$.

3. Figuring out how to keep track of units in computer programming languages is an interesting area for research in computer science.  Currently, Python does not seem to have widely accepted packages for keeping track of time and currency units like phyicists keep track of kilograms, meters, and seconds. Thus, it is necessary to convert many quantities into dimensionless types like np.float64 and keep track of currency or time units non-programatically.

4. With np.datetime64, some ambiguous time calculations are flagged as errors, some ambiguous calculations are not flagged as errors, and some potentially useful calculations are flagged as errors as well. Therefore, I recommend testing np.datetime64 code carefully to make sure it works as expected. Examples:

In [2]:
one_minute = np.timedelta64(1, 'm')
print(f"{one_minute=}")

one_minute_seconds = one_minute.astype('timedelta64[s]')
one_minute_seconds2 = np.timedelta64(one_minute, 's')
print(f"{one_minute_seconds=}: works as expected")
print(f"{one_minute_seconds2=}: works as expected")

dtm = np.timedelta64(1, 'M')
print(f"\n{dtm=} = one month")

dtm_days = dtm.astype('timedelta64[D]')  # convert one month to days
#dtm_days = np.timedelta64(dtm, 'D')  # error, ambiguous?
print(f"{dtm_days=} = one month in days. Is this ambiguous or not?")

one_day = np.timedelta64(1, 'D')
one_year = np.timedelta64(1, 'Y')
one_year_days = one_year.astype('timedelta64[D]')
# days_in_year = np.timedelta64(one_year, 'D')  # error, ambiguous?
# days_in_one_year = one_year / one_day  # error, ambiguous?
print(f"{one_year_days=}: Is this ambiguous or not?")



one_minute=numpy.timedelta64(1,'m')
one_minute_seconds=numpy.timedelta64(60,'s'): works as expected
one_minute_seconds2=numpy.timedelta64(60,'s'): works as expected

dtm=numpy.timedelta64(1,'M') = one month
dtm_days=numpy.timedelta64(30,'D') = one month in days. Is this ambiguous or not?
one_year_days=numpy.timedelta64(365,'D'): Is this ambiguous or not?


## How to use numpy datetime64 effectively

Sometimes it is useful to use numpy date functionality to calculate days between dates, then plug the days into an a formula such as $(1 + r \cdot m)$.  If $r$ is measured per year, the days must be converted into years for dimensions to be consistent.

For example, suppose I want to invest 100 dollars for six months, keeping track of the exact number of days, making sure that the end-date is a business day, calculating simple interest.

Here is one way to do it:

In [3]:
t0 = np.datetime64('2022-08-12', 'D')  #start date
t1 = np.datetime64('2023-02-12', 'D')  # end date, but it might be a weekend or holiday
b_busday = np.is_busday(t1) 
m = t1 - t0  # incorrect time interval ending on weekend or holiday
print(f"{t0=}\n{t1=}\n{m=}\nFalse means t1 is not a business day: {b_busday=}\n")

t1 =  np.busday_offset('2023-02-12', 0, roll='forward')  # Fix t1 by rolling date forward to next business day
b_busday = np.is_busday(t1)
m = t1 - t0  # exact number of days, corrected for weekends and holidays
print(f"{t0=}\n{t1=}\n{m=}\nTrue means t1 is a business day:{b_busday=}\n")

md = m.astype('int') # convert np.datetime64 to np.int64 number of days
days_in_year = 360.00
dt = md / days_in_year  # convert number of days to fraction of a year, using appropriate convention for days in a year.
print(f"Investment period in days: {md=}\n{days_in_year=}\nInvestment period in years: {dt=}\n")

v_0 = 100.00  # dollars
r = 0.06 # interest rate per year
v_1 = v_0 * (1 + r * dt)

print(f"{v_0=}, {v_1=}")

t0=numpy.datetime64('2022-08-12')
t1=numpy.datetime64('2023-02-12')
m=numpy.timedelta64(184,'D')
False means t1 is not a business day: b_busday=False

t0=numpy.datetime64('2022-08-12')
t1=numpy.datetime64('2023-02-13')
m=numpy.timedelta64(185,'D')
True means t1 is a business day:b_busday=True

Investment period in days: md=185
days_in_year=360.0
Investment period in years: dt=0.5138888888888888

v_0=100.0, v_1=103.08333333333333


In [4]:
print("finished")

finished
